# Description

This notebook focuses on visualizing nodes within the Pywr-DRB model along with USGS gauge locations near those nodes.

In [1]:
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd

crs = 4386

ModuleNotFoundError: No module named 'geopandas'

# Data Preparation

Load the model nodes and edges from the `/model_data` folder. 

Create GeoDataFrame objects for each model component. These components include:

- Nodes:
    - Reservoirs (type = 'storage')
    - River gauges (type = 'rivergauge')
    - Model outputs (type = 'output')
    - Relevant USGS gauges 

- Edges:
    - Mainstem 
    - Tributaries
    - Diversions


In [45]:
### Load nodes, edges, and usgs gauge data
nodes = pd.read_csv('./model_components/drb_model_nodes.csv', sep = ',')
edges = pd.read_csv('./model_components/drb_model_edges.csv', sep = ',')
all_usgs_gauges = pd.read_csv(f'./model_components/drb_usgs_gauges.csv', sep = ',')

### Load drb shapefiles
drb_boundary = gpd.read_file('DRB_shapefiles/drb_bnd_polygon.shp').to_crs(crs)
drb_river = gpd.read_file('DRB_shapefiles/delawareriver.shp').to_crs(crs)

# Reservoir data
reservoir_data = pd.read_csv('../model_data/drb_model_istarf_conus.csv', sep = ',')

### TODO: Need to get the long,lat data for gauges of interest. 
usgs_gauges = pd.read_csv('../model_data/drb_model_usgs_data_sources.csv', sep = ',')

# Filter drb gauges
all_gauges = gpd.GeoDataFrame(
    all_usgs_gauges.drop('geometry', axis = 1), geometry=gpd.points_from_xy(all_usgs_gauges.long, all_usgs_gauges.lat, crs = crs))
drb_gauges = gpd.clip(all_gauges, drb_boundary)

In [46]:
# Specify node and edge types to be plotted
plot_node_types = ['storage', 'rivergauge', 'output', 'link', 'catchment']
plot_edge_types = ['mainstem', 'tributary', 'diversion']

# Filter for just types of interest
nodes = nodes[nodes['type'].isin(plot_node_types)]
edges = edges[edges['type'].isin(plot_edge_types)]

# Counts
n_nodes = nodes.shape[0]

In [47]:
### Add descriptions to nodes of interest
nodes['description'] = ""

## Cannonsville
index = nodes[nodes.name == 'reservoir_cannonsville'].index[0]
nodes.at[index, 'description'] = 'Cannonsville Reservoir <br> One of three major NYC reservoirs.'

### Pepacton
index = nodes[nodes.name == 'reservoir_pepacton'].index[0]
nodes.at[index, 'description'] = 'Pepacton Reservoir <br> One of three major NYC reservoirs.'

### Neversink
index = nodes[nodes.name == 'reservoir_neversink'].index[0]
nodes.at[index, 'description'] = 'Neversink Reservoir <br> One of three major NYC reservoirs.'


### Blue marsh
index = nodes[nodes.name == 'reservoir_blueMarsh'].index[0]
nodes.at[index, 'description'] = 'Blue Marsh Reservoir <br> A flood control reservoir operated by ACE.'

### Trenton site
index = nodes[nodes.name == 'outflow_trenton'].index[0]
nodes.at[index, 'description'] = 'Trenton Gauge Site <br> Point of interest for water policy.'


In [48]:
## Separate reservoirs for later color coding
nyc_main = ['reservoir_cannonsville', 'reservoir_neversink', 'reservoir_pepacton']
normal_supplemental = ['reservoir_blueMarsh', 'reservoir_beltzvilleCombined']
emergency = ['reservoir_wallenpaupack', 'reservoir_mongaupeCombined', 'reservoir_fewalter', 'reservoir_nockamixon', 'reservoir_prompton']
consumptive_makeup = ['reservoir_merrillCreek']
docket = ['reservoir_marshCreek']

regulated = nyc_main + normal_supplemental + emergency + consumptive_makeup + docket
all_reservoirs = nodes[nodes['type'] == 'storage'].name.to_list()

s = set(regulated)
unregulated = [x for x in all_reservoirs if x not in s]


# Geo-plotting

The `folium` package is used. 

See the documentation here: https://python-visualization.github.io/folium/

In [49]:
# Initialize the map
start_coords = [40.7, -75]
geomap = folium.Map(location = start_coords, 
                    zoom_start = 7.35,
                   tiles = 'cartodbpositron',
                   control_scale = True)

# Start a feature group for toggle functionality
reservoir_layer = folium.FeatureGroup(name='Reservoirs', show=True)
output_layer = folium.FeatureGroup(name='Outputs & Diversions', show=False)
flow_target_layer = folium.FeatureGroup(name='DRBC Flow Target Locations', show=False)
gauge_layer = folium.FeatureGroup(name='USGS Gauges', show=True)
catchment_layer = folium.FeatureGroup(name='Reservoir Catchments', show=False)
basin_layer = folium.FeatureGroup(name='DRB Boundary', show=True)


In [50]:
# DRB Boundary
for _, r in drb_boundary.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'none',
                                                    'weight': 1,
                                                    'color':'black', 
                                                    'opacity':1,
                                                     'fill_opacity': 0.8,
                                                    'fill': False})
    folium.Popup('Delaware River Basin Boundary', 
                 min_width = 200, 
                 max_width = 200).add_to(geo_j)
    geo_j.add_to(basin_layer)

# Mainstem River
for _, r in drb_river.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'none',
                                                    'weight': 1,
                                                    'color':'black', 
                                                    'opacity':1,
                                                    'fill': 'none'})
    folium.Popup('Delaware River Main Stem', 
                 min_width = 200, 
                 max_width = 200).add_to(geo_j)
    geo_j.add_to(basin_layer)

In [51]:
### Add USGS Gauges
gauge_colors = ['#89CFF0', '#89CFF0']

storage_colors = ["#A9A9A9", "#A9A9A9"]
gauge_colors = ['#78b72c', '#78b72c']
output_colors = ['#b72c78', '#b72c78']
catchment_colors = ["#3186cc", "#3186cc"]
link_colors = ['#5e5e5e', '#5e5e5e']

# Fill opacity
fop = 0.9

popup_width = 300

output_size  = 10
storage_size = 5
catchment_size = 30
link_size = 2

volume_scale = 1/10
max_radius = 30
min_radius = 10

for n in range(drb_gauges.shape[0]):
    coords = [drb_gauges.lat.iloc[n], drb_gauges.long.iloc[n]]
    
    disp = f'Gauge: 0{drb_gauges.site_id.iloc[n]} <br>Start: {drb_gauges.start_date.iloc[n]} <br>End: {drb_gauges.end_date.iloc[n]}'
    
    pop = folium.Popup(disp, min_width = popup_width, max_width = popup_width)
    
    folium.CircleMarker(coords, 
                            popup = pop,
                            fill_color = gauge_colors[1],
                            fill = True,
                            fill_opacity = fop,
                            radius = 3,
                            color = gauge_colors[0]).add_to(gauge_layer)

In [52]:
# Filter edges which do not have corresponding nodes
edges = edges[edges['node1'].isin(nodes.name)]
edges = edges[edges['node2'].isin(nodes.name)]

edges = edges.reset_index()
n_edges = edges.shape[0]


# Link colors and dimensions
tributary_color = '#9fc5e8'
mainstem_color = '#397aa7'
diversion_color = '#f0b237'

tributary_weight = 4
mainstem_weight = 4
diversion_weight = 1

edge_opacity = 0.9

# Add links
for l in range(n_edges):
    coord_1 = [nodes[nodes["name"] == edges["node1"][l]].lat.iloc[0], nodes[nodes["name"] == edges["node1"][l]].long.iloc[0]]
    coord_2 = [nodes[nodes["name"] == edges["node2"][l]].lat.iloc[0], nodes[nodes["name"] == edges["node2"][l]].long.iloc[0]]
    line = [coord_1, coord_2]
    
    edge_type = edges.type.iloc[l]
    
    if edge_type == 'mainstem':
        folium.PolyLine(line, 
                       weight = mainstem_weight,
                       color = mainstem_color,
                       opacity = edge_opacity).add_to(geomap)
    elif edge_type == 'tributary':
        folium.PolyLine(line, 
                       weight = tributary_weight,
                       color = tributary_color,
                       opacity = edge_opacity).add_to(geomap)
    elif edge_type == 'diversion':
        folium.PolyLine(line, 
                       weight = diversion_weight,
                       color = diversion_color,
                       opacity = edge_opacity).add_to(geomap)

In [53]:
# Plot options
plot_gauges = True
plot_outputs = True
plot_catchments = True
plot_storage = True
plot_links = False

# Specify colors [marker edges, fill]
storage_colors = ["#A9A9A9", "#A9A9A9"]
gauge_colors = ['#78b72c', '#78b72c']
output_colors = ['#b72c78', '#b72c78']
catchment_colors = ["#3186cc", "#3186cc"]
link_colors = ['#5e5e5e', '#5e5e5e']

# Fill opacity
fop = 0.9

popup_width = 300

output_size  = 10
storage_size = 5
catchment_size = 30
link_size = 2

volume_scale = 1/10
max_radius = 30
min_radius = 10


# Add nodes
for n in range(n_nodes):
    coords = [nodes.lat.iloc[n], nodes.long.iloc[n]]
    
    disp = nodes['description'].iloc[n]
    if len(disp) == 0:
        disp = nodes.name.iloc[n]
    
    node_type = nodes.type.iloc[n]
    name = nodes.name.iloc[n]
    
    pop = folium.Popup(disp, min_width = popup_width, max_width = popup_width)
        
    if node_type == 'storage':
        
        res_name = nodes.name.iloc[n].split('_')[1]
        volume = reservoir_data[reservoir_data['reservoir'] == res_name]['GRanD_CAP_MCM'].iloc[0]
        if (volume*volume_scale > max_radius):
            s = max_radius
        elif (volume*volume_scale < min_radius):
            s = min_radius
        else:
            s = volume*volume_scale
            
        pop = folium.Popup(f'{disp} <br> Capacity: {volume} MCM', min_width = popup_width, max_width = popup_width)


        folium.RegularPolygonMarker(coords,
                                   popup = pop,
                                   number_of_sides = 8,
                                   radius = s,
                                    weight = 0.75,
                                   fill_color = storage_colors[1],
                                   fill_opacity = fop,
                                   color = storage_colors[0],
                                   rotation = 90).add_to(reservoir_layer)
        
    elif node_type == 'rivergauge':
        if plot_gauges:
            gauge_name = nodes.name.iloc[n]
            if  gauge_name in ['outflow_neversink', 'outflow_cannonsville', 'outflow_pepacton']:
                pass
            elif gauge_name == 'outflow_trenton':
                folium.Marker(coords, 
                          popup = pop,
                          icon=folium.Icon(color="green"),
                            radius =100).add_to(flow_target_layer)
            else:
                folium.Marker(coords, 
                          popup = pop,
                          icon=folium.Icon(color="purple"),
                            radius =60).add_to(flow_target_layer)
        else:
            pass
                
    elif node_type == 'output':
        if plot_outputs:
            if name == 'output_del':
                c = '#EAC100'
            else:
                c = output_colors[1]
            folium.RegularPolygonMarker(coords,
                                       popup = pop,
                                       number_of_sides = 4, 
                                       radius = output_size,
                                       fill_color= c,
                                        fill_opacity = fop,
                                       color = c).add_to(output_layer)
        else:
            pass
        
    elif node_type == 'catchment':
        if plot_catchments:
            folium.CircleMarker(coords, 
                                popup = pop,
                                fill_color = catchment_colors[1],
                                fill = True,
                                fill_opacity = fop,
                                color = catchment_colors[0]).add_to(catchment_layer)
        else:
            pass
    elif node_type == 'link':
        if plot_links:
            folium.CircleMarker(coords, 
                                popup = pop,
                                fill_color = link_colors[1],
                                fill = False,
                                fill_opacity = 0.2,
                                color = link_colors[0],
                               radius = link_size).add_to(geomap)
        else:
            pass

In [54]:
## Add reservoir layers based on regulation level
nyc_reservoir_layer = folium.FeatureGroup(name='NYC', show=True)
normal_reservoir_layer = folium.FeatureGroup(name='Normal Non-NYC', show=False)
emergency_reservoir_layer = folium.FeatureGroup(name='Emergency', show=False)
docket_reservoir_layer = folium.FeatureGroup(name='Docket', show=False)
consumptive_makeup_reservoir_layer = folium.FeatureGroup(name='Consumptive Make-Up', show=False)
#unregulated_reservoir_layer = folium.FeatureGroup(name='Reservoirs', show=True)

## Color codes
nyc_main_color = '#3D9F2A'
normal_supplemental_color = '#03D19C'
emergency_color = '#E38B14'
consumptive_makeup_color = '#6E6E6E'
docket_color = '#FFEB90'
unregulated_color = '#A4A4A4'

s2_opacity = 0.9

# Add nodes
for n in range(n_nodes):
    coords = [nodes.lat.iloc[n], nodes.long.iloc[n]]
    
    disp = nodes['description'].iloc[n]
    if len(disp) == 0:
        disp = nodes.name.iloc[n]
    
    node_type = nodes.type.iloc[n]
    name = nodes.name.iloc[n]
    
    pop = folium.Popup(disp, min_width = popup_width, max_width = popup_width)
        
    if node_type == 'storage':
        
        res_name = nodes.name.iloc[n].split('_')[1]
        volume = reservoir_data[reservoir_data['reservoir'] == res_name]['GRanD_CAP_MCM'].iloc[0]
        if (volume*volume_scale > max_radius):
            s = max_radius
        elif (volume*volume_scale < min_radius):
            s = min_radius
        else:
            s = volume*volume_scale
            
        pop = folium.Popup(f'{disp} <br> Capacity: {volume} MCM', min_width = popup_width, max_width = popup_width)
        
        if name in nyc_main:
            folium.RegularPolygonMarker(coords,
                                       popup = pop,
                                       number_of_sides = 8,
                                       radius = s,
                                       fill_color = nyc_main_color,
                                       fill_opacity = s2_opacity,
                                       color = nyc_main_color,
                                       rotation = 90).add_to(nyc_reservoir_layer)
        elif name in normal_supplemental:
            folium.RegularPolygonMarker(coords,
                                       popup = pop,
                                       number_of_sides = 8,
                                       radius = s,
                                       fill_color = normal_supplemental_color,
                                       fill_opacity = s2_opacity,
                                       color = normal_supplemental_color,
                                       rotation = 90).add_to(normal_reservoir_layer)

        elif name in emergency:
            folium.RegularPolygonMarker(coords,
                                       popup = pop,
                                       number_of_sides = 8,
                                       radius = s,
                                       fill_color = emergency_color,
                                       fill_opacity = s2_opacity,
                                       color = emergency_color,
                                       rotation = 90).add_to(emergency_reservoir_layer)
        elif name in docket:
            folium.RegularPolygonMarker(coords,
                                       popup = pop,
                                       number_of_sides = 8,
                                       radius = s,
                                       fill_color = docket_color,
                                       fill_opacity = s2_opacity,
                                       color = docket_color,
                                       rotation = 90).add_to(docket_reservoir_layer)

        elif name in consumptive_makeup:
            folium.RegularPolygonMarker(coords,
                                       popup = pop,
                                       number_of_sides = 8,
                                       radius = s,
                                       fill_color = consumptive_makeup_color,
                                       fill_opacity = s2_opacity,
                                       color = consumptive_makeup_color,
                                       rotation = 90).add_to(consumptive_makeup_reservoir_layer)

        else:
            pass

In [55]:
# Display the map
reservoir_layer.add_to(geomap)
output_layer.add_to(geomap)
catchment_layer.add_to(geomap)
flow_target_layer.add_to(geomap)
basin_layer.add_to(geomap)
gauge_layer.add_to(geomap)

nyc_reservoir_layer.add_to(geomap)
normal_reservoir_layer.add_to(geomap)
emergency_reservoir_layer.add_to(geomap)
docket_reservoir_layer.add_to(geomap)
consumptive_makeup_reservoir_layer.add_to(geomap)
folium.TileLayer('stamenwatercolor').add_to(geomap)
folium.TileLayer('cartodbdark_matter').add_to(geomap)


folium.LayerControl().add_to(geomap)
geomap

In [44]:
# Save as an HTML
geomap.save("drb_model_map.html")